In [1]:
from tensorflow.python.client import device_lib
#print(device_lib.list_local_devices())

import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

Num GPUs Available:  1


In [2]:
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import BatchNormalization, Conv2D, MaxPooling2D, Flatten, Dense, Input, Dropout 
from sklearn.model_selection import train_test_split

In [4]:
data="data/cleaned_all_data/"

In [5]:
df=pd.read_csv("data/annotated_data.csv")

In [6]:
df.info

<bound method DataFrame.info of                    data  annotation
0       E-S-01-F-01.png           1
1       E-S-01-F-02.png           1
2       E-S-01-F-03.png           1
3       E-S-01-F-04.png           1
4       E-S-01-F-05.png           1
...                 ...         ...
18300  B-S-100-G-20.tif           0
18301  B-S-100-G-21.tif           0
18302  B-S-100-G-22.tif           0
18303  B-S-100-G-23.tif           0
18304  B-S-100-G-24.tif           0

[18305 rows x 2 columns]>

In [7]:
df.head()

,data,annotation
0,E-S-01-F-01.png,1
1,E-S-01-F-02.png,1
2,E-S-01-F-03.png,1
3,E-S-01-F-04.png,1
4,E-S-01-F-05.png,1


In [8]:
images=df["data"].values

In [9]:
len(images)

18305

In [10]:
labels=df["annotation"].values

In [11]:
len(labels)

18305

In [12]:
import cv2
import numpy as np

In [13]:
# train_data=tf.data.Dataset.from_tensor_slices((images,labels))

In [14]:
# def get_images(img,label):
#     image=cv2.imread(data+img,cv2.IMREAD_GRAYSCALE)
#     #image=tf.image.decode_image(image,channels=1,dtype=tf.int64)
#     return image,label

In [15]:
X=[]

In [16]:
for image in images[:]:
    img=cv2.imread(data+image,cv2.IMREAD_UNCHANGED)
    img=cv2.bitwise_not(img)
    X.append(img)

In [17]:
X=np.array(X)

In [18]:
Y=[]
for label in labels:
    label_=[label]
    Y.append(label_)

In [19]:
X.shape

(18305, 256, 512)

In [20]:
Y=np.array(Y)

In [21]:
Y.shape

(18305, 1)

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state=42)

In [23]:
y_train.shape

(14644, 1)

In [24]:
model=Sequential()

In [25]:
model.add(Input((256,512,1)))
# model.add(Conv2D(32,activation = 'relu',kernel_size = (3,3)))
# model.add(MaxPooling2D(pool_size = (2,2)))
#model.add(BatchNormalization())
model.add(Conv2D(64,activation = 'relu',kernel_size = (3,3)))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Conv2D(128,activation = 'relu',kernel_size = (3,3)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Conv2D(256,activation = 'relu',kernel_size = (3,3)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Conv2D(512,activation = 'relu',kernel_size = (3,3)))
model.add(MaxPooling2D(pool_size = (2,2)))
# model.add(BatchNormalization())
model.add(Conv2D(1024,activation = 'relu',kernel_size = (3,3)))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Flatten())
model.add(Dropout(0.35))
model.add(Dense(64, activation = 'relu'))
# model.add(Dropout(0.3))
# model.add(Dense(16, activation = 'relu'))
model.add(Dense(2, activation = "softmax"))

In [26]:
opt = SGD(learning_rate=0.01, momentum=0.9)

In [27]:
model.compile(loss = "sparse_categorical_crossentropy", optimizer = "adam", metrics=['accuracy'])

In [28]:
filename = 'fraud_sign_det_test5_1024con.h5'

In [29]:
checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

In [30]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 254, 510, 64)      640       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 127, 255, 64)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 125, 253, 128)     73856     
                                                                 
 batch_normalization (BatchN  (None, 125, 253, 128)    512       
 ormalization)                                                   
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 62, 126, 128)     0         
 2D)                                                             
                                                        

In [ ]:
history = model.fit(X_train, y_train, validation_data=(X_test,y_test), epochs = 10, batch_size = 12,verbose=1, callbacks=[checkpoint])

In [ ]:
def plot_accuracy(history):
    plt.plot(history.history['val_accuracy'])
    plt.plot(history.history['accuracy'])
#     plt.plot(history.history['val_sparse_categorical_accuracy'])
#     plt.plot(history.history['sparse_categorical_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['val_accuracy', 'Train_accuracy'], loc = 'best')
    plt.show()

In [ ]:
plot_accuracy(history)

In [ ]:
def plot_loss(history):
    plt.plot(history.history['val_loss'])
    plt.plot(history.history['loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['val_loss', 'train_loss'], loc = 'best')
    plt.show()

In [ ]:
plot_loss(history)